# Standard Algorithm for Nearest Neighbours

In [ ]:
# First, we need to install openML
!pip install openml

In [8]:
# Here we get the standard K-Nearest Neighbors
from sklearn import neighbors
from openml import tasks

# Build the k-Neighbors
knn = neighbors.KNeighborsClassifier

In [10]:
import openml

# List all tasks in a benchmark
benchmark = openml.study.get_suite('OpenML-CC18')
tasks.list_tasks(output_format="dataframe", task_id=benchmark.tasks)

# Return benchmark results
openml.evaluations.list_evaluations(
    function="area_under_roc_curve",
    tasks=benchmark.tasks,
    output_format="dataframe"
)

,run_id,task_id,setup_id,flow_id,flow_name,data_id,data_name,function,upload_time,uploader,uploader_name,value,values,array_data
0,28,3,18,77,weka.LogitBoost_DecisionStump(1),3,kr-vs-kp,area_under_roc_curve,2014-04-06 23:47:04,1,Jan van Rijn,0.977253,None,"[0.977253,0.977253]"
1,78,3,3,58,weka.NaiveBayes(1),3,kr-vs-kp,area_under_roc_curve,2014-04-07 00:03:46,1,Jan van Rijn,0.951281,None,"[0.951281,0.951281]"
2,139,3,2,57,weka.OneR(1),3,kr-vs-kp,area_under_roc_curve,2014-04-07 00:46:29,1,Jan van Rijn,0.666985,None,"[0.666985,0.666985]"
3,217,3,7,62,weka.DecisionStump(1),3,kr-vs-kp,area_under_roc_curve,2014-04-07 01:25:07,1,Jan van Rijn,0.666432,None,"[0.666432,0.666432]"
4,221,3,6,61,weka.REPTree(1),3,kr-vs-kp,area_under_roc_curve,2014-04-07 01:27:12,1,Jan van Rijn,0.997912,None,"[0.997912,0.997912]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10046421,3,7971068,8815,sklearn.pipeline.Pipeline(columntransformer=sk...,3,kr-vs-kp,area_under_roc_curve,2019-01-24 23:56:55,1,Jan van Rijn,0.996367,None,"[0.996367,0.996367]"
9996,10046459,3,7971107,8815,sklearn.pipeline.Pipeline(columntransformer=sk...,3,kr-vs-kp,area_under_roc_curve,2019-01-24 23:57:03,1,Jan van Rijn,0.997402,None,"[0.997402,0.997402]"
9997,10046499,3,7971148,8815,sklearn.pipeline.Pipeline(columntransformer=sk...,3,kr-vs-kp,area_under_roc_curve,2019-01-24 23:57:16,1,Jan van Rijn,0.997317,None,"[0.997317,0.997317]"
9998,10046616,3,7971269,8815,sklearn.pipeline.Pipeline(columntransformer=sk...,3,kr-vs-kp,area_under_roc_curve,2019-01-24 23:57:48,1,Jan van Rijn,0.996206,None,"[0.996206,0.996206]"


# Source:
- https://www.openml.org/apis
- https://www.openml.org/search?type=study&sort=tasks_included&study_type=task&id=99

@article{oml-benchmarking-suites,
      title={OpenML Benchmarking Suites}, 
      author={Bernd Bischl and Giuseppe Casalicchio and Matthias Feurer and Frank Hutter and Michel Lang and Rafael G. Mantovani and Jan N. van Rijn and Joaquin Vanschoren},
      year={2019},
      journal={arXiv:1708.03731v2 [stat.ML]}
}